# Data Preparation

### Prepare the space

Install/import libraries, read the excel file, convert to .csv

In [1]:
# Install openpyxl and import pandas
!pip install openpyxl
import pandas as pd

# Read the excel file using the openpyxl engine
file = 'What_We_Watched_A_Netflix_Engagement_Report_2023Jan-Jun.xlsx'
dataframe = pd.read_excel(file, engine='openpyxl')

# Convert it to .csv
filename = 'What_We_Watched_A_Netflix_Engagement_Report_2023Jan-Jun.csv'
dataframe.to_csv(filename, index=False)


[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


Inspect .csv

In [2]:
# Let's load the dataframe again but this time skip the first 5 rows
df = pd.read_csv('What_We_Watched_A_Netflix_Engagement_Report_2023Jan-Jun.csv', skiprows=5)

# Disconsidering first five lines, let's look at the structure again
df.head()

,Unnamed: 0,Title,Available Globally?,Release Date,Hours Viewed
0,NaN,The Night Agent: Season 1,Yes,2023-03-23 00:00:00,812100000
1,NaN,Ginny & Georgia: Season 2,Yes,2023-01-05 00:00:00,665100000
2,NaN,The Glory: Season 1 // 더 글로리: 시즌 1,Yes,2022-12-30 00:00:00,622800000
3,NaN,Wednesday: Season 1,Yes,2022-11-23 00:00:00,507700000
4,NaN,Queen Charlotte: A Bridgerton Story,Yes,2023-05-04 00:00:00,503000000


### Create / Manipulate Data

Create a column of data based on the Title field that shows whether a Title is English language or foreign based on the presence of foreign characters in the Title. This will not be a 100% accurate column for all 18k rows, but inspecting the data shows that this pattern will have a high enough degree of accuracy to use for broad analysis.

In [3]:
import string

# Define new function to determine whether a title is English or Foreign
def western_or_other_v2(title):
    allowed_chars = string.ascii_letters + string.digits + string.punctuation + ' '
    return 'English' if all(char in allowed_chars for char in title) else 'Foreign'

# Apply our function to the Title column and store the result in a new column called "Western / Other"
df['English / Foreign'] = df['Title'].apply(western_or_other_v2)

# Inspecting some rows of the data with the new column
df.head(10)

,Unnamed: 0,Title,Available Globally?,Release Date,Hours Viewed,English / Foreign
0,NaN,The Night Agent: Season 1,Yes,2023-03-23 00:00:00,812100000,English
1,NaN,Ginny & Georgia: Season 2,Yes,2023-01-05 00:00:00,665100000,English
2,NaN,The Glory: Season 1 // 더 글로리: 시즌 1,Yes,2022-12-30 00:00:00,622800000,Foreign
3,NaN,Wednesday: Season 1,Yes,2022-11-23 00:00:00,507700000,English
4,NaN,Queen Charlotte: A Bridgerton Story,Yes,2023-05-04 00:00:00,503000000,English
5,NaN,You: Season 4,Yes,2023-02-09 00:00:00,440600000,English
6,NaN,La Reina del Sur: Season 3,No,2022-12-30 00:00:00,429600000,English
7,NaN,Outer Banks: Season 3,Yes,2023-02-23 00:00:00,402500000,English
8,NaN,Ginny & Georgia: Season 1,Yes,2021-02-24 00:00:00,302100000,English
9,NaN,FUBAR: Season 1,Yes,2023-05-25 00:00:00,266200000,English


Create a new column reflecting whether the Title is a TV show or Movie based on Title. Nearly every TV show contains the phrase "Season _" in it. This method ignores the presence of mini-series, which are only intended to have 1 season so do not have a Season marker in the title. To mitigate this oversight, I added an additional criteria to assume a title is a TV show if it has more views than the highest viewed movie of all time on Netflix. I.e. if it does not have the phrase "Season _" in the title, but has been viewed more than the most viewed movie of all time (Red Notice with 68,100,000 hours viewed), then it must be a TV show.

In [4]:
# Importing the regular expression module
import re

# Define function to categorize as 'TV' or 'Movie'
def tv_or_movie(row):
    # Check if "Season " followed by a number is in the title
    if re.search(r'Season \d', row['Title']) or row['Hours Viewed'] > 68100000:
        return 'TV'
    else:
        return 'Movie'

# Apply our function to the DataFrame and store the result in a new column called "TV / Movie"
df['TV / Movie'] = df.apply(tv_or_movie, axis=1)

# Inspecting some rows of the data with the new column
df.head(10)

,Unnamed: 0,Title,Available Globally?,Release Date,Hours Viewed,English / Foreign,TV / Movie
0,NaN,The Night Agent: Season 1,Yes,2023-03-23 00:00:00,812100000,English,TV
1,NaN,Ginny & Georgia: Season 2,Yes,2023-01-05 00:00:00,665100000,English,TV
2,NaN,The Glory: Season 1 // 더 글로리: 시즌 1,Yes,2022-12-30 00:00:00,622800000,Foreign,TV
3,NaN,Wednesday: Season 1,Yes,2022-11-23 00:00:00,507700000,English,TV
4,NaN,Queen Charlotte: A Bridgerton Story,Yes,2023-05-04 00:00:00,503000000,English,TV
5,NaN,You: Season 4,Yes,2023-02-09 00:00:00,440600000,English,TV
6,NaN,La Reina del Sur: Season 3,No,2022-12-30 00:00:00,429600000,English,TV
7,NaN,Outer Banks: Season 3,Yes,2023-02-23 00:00:00,402500000,English,TV
8,NaN,Ginny & Georgia: Season 1,Yes,2021-02-24 00:00:00,302100000,English,TV
9,NaN,FUBAR: Season 1,Yes,2023-05-25 00:00:00,266200000,English,TV


Drop of a column full of nan, and overwrite of the old version of the .csv (the final line can be replaced with an export of the .csv in a different environment).

In [5]:
df.drop(columns=['Unnamed: 0'], inplace=True)
df.to_csv(filename, index=False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=240a32a9-e6ee-4b04-8c51-982daa15afa9' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>